In [2]:
# !pip install -q ollama langchain_google_genai dotenv pandas

^C


In [3]:
import ollama
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
import pandas as pd

In [8]:
def read_file(filename):
    file = open(filename,'r')
    str = file.read()
    return str

In [9]:
def get_custom_prompt(question,sql_file):
    # schema = open(sql_file,'r').read()


    prompt = f""" 
    ### Instructions:
    Your task is to convert a question into a SQL query, given a Postgres database schema.
    Adhere to these rules:
    - **Deliberately go through the question and database schema word by word** to appropriately answer the question
    - **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
    - When creating a ratio, always cast the numerator as float

    ### Input:
    Generate a SQL query that answers the question `{question}`.
    
    This query will run on a database whose schema is represented in this string:

    {read_file(sql_file)}

    ### Response:
    Based on your instructions, here is the SQL query I have generated to answer the question `{question}` without any additional text.:
    ```sql
    """
    return prompt

In [10]:
def get_rate_prompt(schema,question,answer):
    prompt = f"""

    ### Instructions:
    Your Task is to throughly study the given SQL Schema, and the text question asked based on the schema. you will be also be given a SQL query, which you have to rate on scale of 10.
    
    Adhere to these rules:
    - **Deliberately go through the question and database schema word by word** to appropriately answer the question
    - **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
    - When creating a ratio, always cast the numerator as float


    ## Input:
    A question, {question}, is asked based on schema as given below.
    
    Overall Rate the SQL query on scale of 10, `{answer}`, based on the efficiency simplicity.
    This Query is suppose to run on the schema in the following string:

    {read_file(schema)}

    I DONT WANT EXPLAINATION FOR YOU RATING! I ONLY WANT THE RATING! THATS IT! JUST NUMBER NOT EVEN 10!

    

    ### Response:
    Based on your instructions, here is the rating of the sql query for provided schema without any additional text.:
    """

    return prompt



def get_ratings(schema,question,answer):
    response = llm_gemini.invoke(
        get_rate_prompt(
            schema = schema,
            question = question,
            answer = answer,
            )
    )
    
    return response.content

In [10]:
load_dotenv()

llm_gemini = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    api_key = os.getenv("GEMINI_API_KEY")
)


In [ ]:
# answer = llm_gemini.invoke('todays date')
# answer.content


'Today is October 26, 2023.'

In [11]:
# response = get_ratings(schema,question,answer)

In [ ]:
questions = [
    "List all drugs in the 'Painkillers' category with a stock below 100 units, along with their supplier names and the date when their inventory was last updated. Order results by oldest inventory update first.",
    "Identify the top 5 products by total sales amount in the last quarter of 2024, including their category names and the store locations where they were sold. Order the results by total sales amount in descending order.",
    "Find the top 3 products with the highest average rating from user reviews, including their category names and total sales amounts. Only consider products that have been ordered at least 10 times. Order the results by average rating in descending order.",
    "List all agents who have facilitated sales of properties with a total sale price exceeding $500,000 in the last year. Include their full names, the number of properties sold, and the total commission earned. Order the results by total commission in descending order.",
    "Identify the top 5 products by total sales amount in the last quarter of 2024, including the name of the employee who made the sale and the total quantity sold for each product. Order the results by total sales amount in descending order.",
    "Retrieve the first name of customers who have given a 5-star rating to a restaurant and have also placed at least one order at that restaurant. For each such customer-restaurant pair, also retrieve the name of the restaurant and the total amount the customer has spent at that restaurant.",
    "Retrieve the first and last name of customers who have purchased a vehicle and also requested a service for that same vehicle, along with the model of the vehicle and the total cost of all services for that vehicle.",
    "How can you retrieve a list of all active subscriptions along with the customer's name, plan name, and the subscription's start and end dates",
    "How can you retrieve a report showing the total stock level of each product across all warehouses, including the product name and total quantity available?",
    "How can you generate a report showing the total sales revenue for each salesperson, including their name and the total amount sold?",
]

responses = []
ratings = []

In [ ]:
model_list = ['qwen2.5-coder:7b','gemma3:4b','deepseek-coder:6.7b']


for model in model_list:
    for i in range(5):
        FILENAME = f"table_list{i+1}_exp.sql"

        response = ollama.chat(model=model, messages=[
        {
            'role': 'user',
            # 'content': get_prompt("What are the names and prices of all drugs in the 'Antibiotics' category?")
            'content': get_custom_prompt(
                question = questions[i],
                sql_file = FILENAME
            )
            # 'content': get_prompt("Which pharmacies have sold more than 100 units of drugs from the 'Painkillers' category in the last 30 days, and what is the total revenue generated from those sales, including the names of the employees who made the sales?")
        },
        ])

        res = response['message']['content']

        responses.append(res)

        rating = get_ratings(
            schema = f"table_list{i+1}_exp.sql",
            question = questions[i],
            answer = res,
        )

        ratings.append(rating)




In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'